In [87]:
from vectice import connect, S3Resource, Dataset, Model
from boto3 import client 
from botocore import UNSIGNED
from botocore.client import Config

s3_client = client('s3', config=Config(signature_version=UNSIGNED), region_name='us-west-1')
users_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/users.csv")
comp_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/competitor_info.csv")
clean_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/competitor_info.csv")
train_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/traindataset.csv")
test_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/testdataset.csv")
val_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/validatedataset.csv")

In [88]:
vtc_prj = connect(config="vectice-config.json")

Welcome, Eric Barre. You`re now successfully connected to the project Sample Project in Vectice.

To access a specific phase, use project.phase(Phase ID)
To get a list of phases you can access and their IDs, use project.list_phases()

For quick access to the list of phases in the Vectice web app, visit:
https://app.vectice.com/project?w=1930&resourceId=8052


In [89]:
# List out the phases in the project
vtc_prj.list_phases()

There are 2 phases in the project Sample Project for user Eric Barre.

**Legend**     [C]  Completed     [IP] In Progress
               [IR] In Review     [NS] Not Started



 phase id           ┃ name                       ┃ owner           ┃ status         ┃ iterations     ┃ steps            
━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━
 34414              │ DataPrep                   │ Unassigned      │ [NS]           │ 0/0            │ 3                
 34415              │ Modeling                   │ Unassigned      │ [NS]           │ 0/0            │ 3


>> To access a specific phase, use project.phase(PhaseID)
>> To access an array of phases, use project.phases

For quick access to the list of phases for this project, visit:
https://app.vectice.com/project?w=1930&resourceId=8052



In [90]:
vtc_phase = vtc_prj.phase("DataPrep")

In [91]:
vtc_phase.list_step_definitions()

There are 3 step definitions in the phase DataPrep for user Eric Barre.



 step def. id  ┃ name                   ┃ description                      ┃ shortcut                                   
━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
 11307         │ 1. Data Collection     │ Identify suitable datasets for … │ step_data_collection                       
 11308         │ 2. EDA                 │ Normalize, clean, join data to … │ step_eda                                   
 11309         │ 3. Split dataset       │ Split the datasets into trainin… │ step_split_dataset


For quick access to the list of step definitions in the Vectice web app, visit:
https://app.vectice.com/project/phase/steps?w=1930&pageId=34414



In [92]:
# Document the project origin datasets
vtc_iter = vtc_phase.iteration()
vtc_iter.step_data_collection = Dataset.origin( name="Users", resource=users_ds)
vtc_iter.step_data_collection += Dataset.origin( name="Competitor_Info", resource=comp_ds)

Successfully registered Dataset(name='Users', id=23979, version='Version 1', type=ORIGIN).
Successfully added Dataset(name='Users', id=23979, version='Version 1', type=MODELING) to Data Collection
Successfully registered Dataset(name='Competitor_Info', id=23980, version='Version 1', type=ORIGIN).
Successfully added Dataset(name='Competitor_Info', id=23980, version='Version 1', type=ORIGIN) to Data Collection


In [93]:
# EDA on the datasets
...

# Document our EDA process
vtc_iter.step_eda = Dataset.clean(name="Clean Dataset",resource=clean_ds, derived_from=[23979, 23980])


# Split datasets for model and register them
...

vtc_iter.step_split_dataset = Dataset.modeling(name="Modeling Dataset", training_resource=train_ds,testing_resource=test_ds,validation_resource=val_ds)


# All steps on the phase are completed, mark the iteration as complete
vtc_iter.complete()

Successfully registered Dataset(name='Clean Dataset', id=23981, version='Version 1', type=CLEAN).
Successfully added Dataset(name='Clean Dataset', id=23981, version='Version 1', type=MODELING) to EDA
Successfully registered Dataset(name='Modeling Dataset', id=23982, version='Version 1', type=MODELING).
Successfully added Dataset(name='Modeling Dataset', id=23982, version='Version 1', type=MODELING) to Split dataset


In [95]:
# Get the next phase (modeling)
vtc_iter = vtc_prj.phase(34415).iteration()

# Build the model
...

# Document the model technique used
vtc_iter.step_document_modeling_technique = "Using a Linear Regression model."

# Connecting your model to an iteration will automatically sync the information to Vectice UI
vtc_iter.step_build_model = Model(library="sklearn", 
                                      technique="ln", 
                                      attachments=["Feature_Importance.png", "26_LiftChart_holdout.png"], 
                                      name="Base_LR_Model")


# Fine tune hyper-parameters
...

# Register model in Vectice
vtc_iter.step_fine_tune_model = Model(library="sklearn", 
                                      technique="ln", 
                                      attachments=["Feature_Importance.png", "26_LiftChart_holdout.png"], 
                                      name="Tuned_LR_Model",
                                      metrics={"metric_1": 1, "metric_2": 1}, 
                                      properties={"prop": "value"})

# All steps on the phase are completed, mark the iteration as complete
vtc_iter.complete()

Attachments with names: ['26_LiftChart_holdout.png', 'Feature_Importance.png'] successfully attached to Model(name='Base_LR_Model', version='Version 2').
Successfully registered Model(name='Base_LR_Model', library='sklearn', technique='ln', version='Version 2').
Model: Base_LR_Model already exists.
Successfully added Model(name='Base_LR_Model', version='Version 2') to Build Model
Successfully added Attachment: 41651 from Model(name='Base_LR_Model', version='Version 2') to Build Model
Successfully added Attachment: 41652 from Model(name='Base_LR_Model', version='Version 2') to Build Model
Attachments with names: ['26_LiftChart_holdout.png', 'Feature_Importance.png'] successfully attached to Model(name='Tuned_LR_Model', version='Version 1').
Successfully registered Model(name='Tuned_LR_Model', library='sklearn', technique='ln', version='Version 1').
Successfully added Model(name='Tuned_LR_Model', version='Version 1') to Fine tune model
Successfully added Attachment: 41653 from Model(name